In [ ]:
from concurrent.futures import ProcessPoolExecutor
from pokerkit import *
import random
with ProcessPoolExecutor() as executor:
    values = calculate_equities(
        (
            parse_range('AK'),
            parse_range('22'),
        ),
        (),
        2,
        5,
        Deck.STANDARD,
        (StandardHighHand,),
        sample_count=10000,
        executor=executor,
    )


RANKS = '23456789TJQKA'
SUITS = 'shdc'
DECK = [r + s for r in RANKS for s in SUITS]

def deal_hands_fast():
    # random.sample is highly optimized in C for picking without replacement
    s = random.sample(DECK, 4)
    
    # f-strings are generally faster than "".join() for small sets
    return f"{s[0]}{s[1]}", f"{s[2]}{s[3]}"
hand1, hand2 = deal_hands_fast()
print(hand1)
print(hand2)


2c8h
4c2h


In [42]:
# Top 20% of hands in strict descending order of raw equity
# Cumulative combinations: 266
top_20_equity_ordered = [
    'AA', 'KK', 'QQ', 'JJ', 'TT', '99', '88', 'AKs', '77', 'AQs', 
    'AJs', 'AKo', 'ATs', 'AQo', '66', 'AJo', 'KQs', 'A9s', 'ATo', 'A8s', 
    'KJs', 'A7s', 'KQo', 'A5s', 'A6s', 'A4s', '55', 'A3s', 'QJs', 'A2s', 
    'KTs', 'A9o', 'KJo', '44', 'QTs', 'A8o', 'K9s', '33', 'KTo', 'Q9s', 
    'QJo', 'JTs'
]
playable_hands = top_20_equity_ordered[:10]
print(playable_hands)
# Map ranks to values for easy comparison
RANK_VALS = {r: i for i, r in enumerate('23456789TJQKA')}

def get_shorthand(hand):
    # hand is a 4-char string like "2c8h"
    r1, s1, r2, s2 = hand[0], hand[1], hand[2], hand[3]
    
    # Sort by rank value so the higher card is always first
    if RANK_VALS[r1] >= RANK_VALS[r2]:
        shorthand = r1 + r2
    else:
        shorthand = r2 + r1
        
    # Check if suited or offsuit (ignored for pairs)
    if r1 == r2:
        return shorthand # Returns "AA", "88", etc.
    elif s1 == s2:
        return shorthand + 's'
    else:
        return shorthand + 'o'

# Example usage:
print(get_shorthand("2c8h")) 
print(get_shorthand("AsKs")) 

['AA', 'KK', 'QQ', 'JJ', 'TT', '99', '88', 'AKs', '77', 'AQs']
82o
AKs


In [91]:
with ProcessPoolExecutor() as executor:
    value = calculate_equities(
                    (
                        parse_range('AA'),
                        parse_range('22'),
                    ),
                    (),
                    2,
                    5,
                    Deck.STANDARD,
                    (StandardHighHand,),
                    sample_count=1,
                    executor=executor,
                )[0]
print(value)

1.0


In [130]:
win_list = []
with ProcessPoolExecutor() as executor:
    for i in range(len(top_20_equity_ordered)):
        playable_hands = top_20_equity_ordered[:i]
        blind = 1
        pure_player = 200 #number of small blinds
        pro_player = 200
        wins = 0
        for _ in range(10000):
            pure_player = 200 #number of small blinds
            pro_player = 200
            while 0 < pro_player < 400:
                blind *= -1
                pure_hand, pro_hand = deal_hands_fast()
                if get_shorthand(pro_hand) not in playable_hands:
                    if blind == 1:
                        pure_player += 2
                        pro_player -= 2
                    else: 
                        pure_player += 1
                        pro_player -= 1
                    
                else:
                    
                    value = calculate_equities(
                            (
                                parse_range(pro_hand),
                                parse_range(pure_hand),
                            ),
                            (),
                            2,
                            5,
                            Deck.STANDARD,
                            (StandardHighHand,),
                            sample_count=1,
                            executor=executor,
                        )[0]
                    if value == 1:
                        if pro_player >= pure_player:
                            pro_player = 400
                        else:
                            pro_player = pro_player * 2
                            pure_player = pure_player - pro_player
                    else: 
                        if pro_player <= pure_player:
                            pro_player = 0
                        else:
                            pro_player = pro_player - pure_player
                            pure_player = pure_player * 2
            if pro_player == 400:
                wins += 1
        
        win_list.append(wins)
        
       



In [135]:
print(win_list)


[0, 966, 2617, 4149, 5200, 5885, 6329, 6341, 6256, 6265, 6173, 5993, 5711, 5677, 5454, 5320, 5227, 5080, 5120, 5026, 4955, 4955, 4870, 4800, 4811, 4697, 4723, 4618, 4692, 4567, 4554, 4603, 4513, 4466, 4573, 4538, 4453, 4591, 4519, 4500, 4421, 4442]


In [136]:
max_index = win_list.index(max(win_list))
print(max_index)
print(top_20_equity_ordered[:max_index + 1])

7
['AA', 'KK', 'QQ', 'JJ', 'TT', '99', '88', 'AKs']
